In [ ]:
'''This script is used to extract the entropy values from the output files of Truncated NMA.'''
import os
import numpy as np
root_dir = "/Directory"
forcefields = ["nmode_data_gaff"]
molecules = ["4ntj_azj", "4ntj_psb", "4ntj_tiq", "4pxz", "4py0"]
suffixes = ["", "_muta"]
methods = ["STM","MTM"]

#For STM dataset
def extract_delta_total_stm(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            if "DELTA S total=" in line:
                return float(line.split()[3]) * 298 / 298.15  # The calculation temperature is set to 298K as default, but experimental temperature is 298.15K
    return None

results = {}

#For MTM dataset
def extract_delta_total_mtm(filepath):
    with open(filepath, 'r') as f:
        found_differences_section = False
        for line in f:
            if "Differences" in line:
                found_differences_section = True
                continue
            if found_differences_section and "Total" in line:
                return float(line.split()[1]) * 298 / 298.15  # The calculation temperature is set to 298K as default, but experimental temperature is 298.15K  
    return None

for forcefield in forcefields:
    for molecule in molecules:
        for suffix in suffixes:
            group = f"{molecule}{suffix}"
            for method in methods:
                data_dir = os.path.join(root_dir, forcefield, group, method)

                if not os.path.exists(data_dir):
                    print(f"Warning: {data_dir} does not exist.")
                    continue

                delta_totals = []
                for filename in os.listdir(data_dir):
                    if filename.endswith(".dat"):
                        filepath = os.path.join(data_dir, filename)
                        if method == "STM":
                            delta = extract_delta_total_stm(filepath)
                        else:
                            delta = extract_delta_total_mtm(filepath)
                        if delta is not None:
                            delta_totals.append(delta)

                if delta_totals:
                    avg_delta = np.mean(delta_totals)
                else:
                    avg_delta = None

                key = f"{forcefield}_{group}_{method}"
                results[key] = avg_delta

values_list = []
for key, value in results.items():
    print(f"{key}: {value}")
    values_list.append(round(float(value),2)) 
print(values_list)
output_file = os.path.join(root_dir, "entropy_results.txt")
with open(output_file, 'w') as f:
    for key, value in results.items():
        f.write(f"{key}: {value}\n")
